# 配置环境

In [ ]:
!git clone https://github.com/tiaoteek/fasterRCNN
%cd fasterRCNN
%pip install -qr requirements.txt

import torch
from IPython.display import Image, clear_output  # to display images
clear_output()
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 1.10.0+cu111 (Tesla K80)


In [ ]:
import torchvision  # 0.3.0  version  这里指的是所使用包的版本
from torchvision import transforms as T
import cv2  # 4.1.1  version
import matplotlib.pyplot as plt  # 3.0.0  version
from PIL import Image  # 5.3.0  version
import random
import os
import torch
import numpy as np

# 加载模型

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True).to(device)  #  加载模型
model.eval()  # 设置成评估模式
clear_output()

预测结果

In [ ]:
#  获取单张图片的预测结果
def prediction(img_path, file_id):
    img = Image.open(img_path)  # Load the image  加载图片
    transform = T.Compose([T.ToTensor()]) # Defing PyTorch Transform
    img = transform(img)  # Apply the transform to the image  转换成 torch 形式
    pred = model([img.to(device)])  # Pass the image to the model  开始推理
    pred_class = [i for i in pred[0]['labels'].cpu().numpy()]  # Get the Prediction Score  获取预测的类别
    pred_boxes = [[float(i[0]), float(i[1]), float(i[2]-i[0]), float(i[3]-i[1])] for i in pred[0]['boxes'].detach().cpu().numpy()]  # Bounding boxes  获取各个类别的边框
    pred_score = pred[0]['scores'].cpu().detach().numpy()  #  获取各个类别的分数
    img_obj = [{"image_id":file_id,"category_id":int(pred_class[i]), "bbox": pred_boxes[i], "score":float(pred_score[i])} for i in range(len(pred_class))]
    return img_obj

准备数据

In [ ]:
# Download COCO val
torch.hub.download_url_to_file('https://ultralytics.com/assets/coco2017val.zip', 'tmp.zip')
!unzip -q tmp.zip -d ../datasets && rm tmp.zip

  0%|          | 0.00/780M [00:00<?, ?B/s]

In [ ]:
#读取coco数据集里的文件索引
import json
file_dic = {}
with open('/content/datasets/coco/annotations/instances_val2017.json','r',encoding='utf8')as fp:
    json_data = json.load(fp)
    img_arr = json_data["images"]
    for i in range(len(img_arr)):
        img_name = img_arr[i]["file_name"]
        img_id = img_arr[i]["id"]
        file_dic[img_id] = img_name

coco_img_dir = "/content/datasets/coco/images/val2017/"

In [ ]:
# 用cocoval 进行验证结果
result_list = []    #临时保存结果
idx = 1
for file_id in file_dic:
    clear_output()
    coco_img_path = os.path.join(coco_img_dir, file_dic[file_id])
    print("第"+ str(idx) +"个图片")
    list = prediction(coco_img_path, file_id)
    result_list += list
    idx += 1

第5000个图片


In [ ]:
# 保存结果
with open('val2017coco.json','a',encoding='utf8')as fp:
    json.dump(result_list, fp)

In [ ]:
!python mAP.py

loading annotations into memory...
Done (t=0.57s)
creating index...
index created!
4996
Loading and preparing results...
DONE (t=1.64s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=49.02s).
Accumulating evaluation results...
DONE (t=6.20s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.369
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.585
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.396
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.212
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.403
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.482
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.307
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.485
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | m